In [1]:
# Gurobi Solving
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [2]:
df = pd.read_csv('msci_332_group_slopes.csv')
df.head()

,Short Form Group,Group (FSS | Level | Bucket),y_int,slope,adjusted_slope,demand_function
0,0-100-EW-1,0 | 100s (East and West) | 1,3100,-0.390801,-0.390801,3100.0 - 0.391p
1,0-100-EW-2,0 | 100s (East and West) | 2,3100,-0.839589,-0.839589,3100.0 - 0.84p
2,0-100-EW-3,0 | 100s (East and West) | 3,3100,-1.144578,-1.144578,3100.0 - 1.145p
3,0-100-EW-4,0 | 100s (East and West) | 4,3100,-1.691610,-1.691610,3100.0 - 1.692p
4,0-100-EW-5,0 | 100s (East and West) | 5,3100,-2.753346,-2.753346,3100.0 - 2.753p


In [3]:
df['y_int'] = df['y_int'].astype(int)
df['slope'] = df['adjusted_slope'].astype(int)

In [4]:
group_yint = {row['Short Form Group']: row['y_int'] for index, row in df.iterrows()}
group_slope = {row['Short Form Group']: row['adjusted_slope'] for index, row in df.iterrows()}

In [5]:
df.dtypes

Short Form Group                 object
Group (FSS | Level | Bucket)     object
y_int                             int32
slope                             int32
adjusted_slope                  float64
demand_function                  object
dtype: object

In [6]:
groups = list(df['Short Form Group'].values)
TOTAL_SEATS = 19800

In [7]:
m = gp.Model("raptors-tickets-price-optimization")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-09


In [8]:
p = m.addVars(groups, name = "p", lb=0)
d = {row['Short Form Group']: (row['y_int'] + row['slope'] * p[row['Short Form Group']]) for index, row in df.iterrows()}
# demand_functions = {group: lambda x, g=group: df.loc[g, 'y_int'] + df.loc[g, 'adjusted_slope'] * x for group in groups}
# demand_functions

In [9]:
m.update()
m

<gurobi.Model Continuous instance raptors-tickets-price-optimization: 0 constrs, 50 vars, Parameter changes: Username=(user-defined)>

In [10]:
m.setObjective(sum((group_yint[group] - group_slope[group]*(p[group])) * p[group] for group in groups), GRB.MAXIMIZE)
m.update()
m

<gurobi.Model Continuous instance raptors-tickets-price-optimization: 0 constrs, 50 vars, Parameter changes: Username=(user-defined)>

In [11]:
m.addConstrs((p[group] <= group_yint[group] / group_slope[group]) for group in groups)
m.update()
m

<gurobi.Model Continuous instance raptors-tickets-price-optimization: 50 constrs, 50 vars, Parameter changes: Username=(user-defined)>

In [12]:
m

<gurobi.Model Continuous instance raptors-tickets-price-optimization: 50 constrs, 50 vars, Parameter changes: Username=(user-defined)>

In [15]:
m.Params.NonConvex = 2
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: AMD Ryzen 7 4800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 50 rows, 50 columns and 50 nonzeros
Model fingerprint: 0x5967baf4
Model has 50 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+02, 6e+03]
  QObjective range [3e-03, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+02, 2e+05]
Presolve time: 0.01s

Barrier solved model in 0 iterations and 0.01 seconds (0.00 work units)
Model is infeasible or unbounded


In [14]:
constrs = m.getVars()
for constr in constrs:
    print(constr)

<gurobi.Var p[0-100-EW-1]>
<gurobi.Var p[0-100-EW-2]>
<gurobi.Var p[0-100-EW-3]>
<gurobi.Var p[0-100-EW-4]>
<gurobi.Var p[0-100-EW-5]>
<gurobi.Var p[0-100-NS-1]>
<gurobi.Var p[0-100-NS-2]>
<gurobi.Var p[0-100-NS-3]>
<gurobi.Var p[0-100-NS-4]>
<gurobi.Var p[0-100-NS-5]>
<gurobi.Var p[0-300-EW-1]>
<gurobi.Var p[0-300-EW-2]>
<gurobi.Var p[0-300-EW-3]>
<gurobi.Var p[0-300-EW-4]>
<gurobi.Var p[0-300-EW-5]>
<gurobi.Var p[0-300-NS-1]>
<gurobi.Var p[0-300-NS-2]>
<gurobi.Var p[0-300-NS-3]>
<gurobi.Var p[0-300-NS-4]>
<gurobi.Var p[0-300-NS-5]>
<gurobi.Var p[0-courtside-1]>
<gurobi.Var p[0-courtside-2]>
<gurobi.Var p[0-courtside-3]>
<gurobi.Var p[0-courtside-4]>
<gurobi.Var p[0-courtside-5]>
<gurobi.Var p[1-100-EW-1]>
<gurobi.Var p[1-100-EW-2]>
<gurobi.Var p[1-100-EW-3]>
<gurobi.Var p[1-100-EW-4]>
<gurobi.Var p[1-100-EW-5]>
<gurobi.Var p[1-100-NS-1]>
<gurobi.Var p[1-100-NS-2]>
<gurobi.Var p[1-100-NS-3]>
<gurobi.Var p[1-100-NS-4]>
<gurobi.Var p[1-100-NS-5]>
<gurobi.Var p[1-300-EW-1]>
<gurobi.Var p